In [1]:
import numpy as np
import PyPDF2
from nltk.corpus import stopwords
import re

In [2]:
# import textract

In [3]:
# !pip3 install PyPDF2

In [4]:
# !pip3 install textract

### Pdf extact text

In [5]:
pdf_obj = open("One_Indian_Girl_-_Chetan_Bhagat-Redicals.pdf","rb")

In [6]:
reader = PyPDF2.PdfFileReader(pdf_obj)

In [7]:
No_of_pg = reader.numPages
print(No_of_pg)
pg_obj = reader.getPage(50)
pg_obj.extractText()

232


''

In [8]:
# text = ''
# for i in range(232):
# #     pg_obj = reader.getPage(i)
#     text = text + pg_obj.extractText()
# #     print(text)
# text

## PreProcessng Text File 

In [9]:
pdf_obj.close()

In [10]:
f = open("./One_Indian_Girl_-_Chetan_Bhagat-Redicals.txt")

In [11]:
text = f.read()

In [12]:
f.close()

In [13]:
print(len(text))
print(text[:500])
type(text)

464927
 

ONE
INDIAN
GIRL

Chetan Bhagat is the author of six bestselling novels—Fl've Point Someone (2004), One Night @ the
Call Center (2005), The 3Mislakes ofMy Life (2008), 2 States (2009), Revolution 2020 (2011) and
HalfGirlfriend (2014)—which have sold over ten million copies and have been translated into over
a dozen languages worldwide.

In 2008, The New York Times called him ‘the biggest—selling author in India’s history’, a
position he has maintained to date. Almost all his books have been ad


str

In [14]:
# text = text.lower()
# text = re.sub("[^A-Za-z]+"," ",text)#^[a-zA-Z] means any a-z or A-Z at the start of a line
# text = text.split()   #[^a-zA-Z] means any character that IS NOT a-z OR A-Z

## Normalise Document

In [15]:
import nltk

tknizer = nltk.WordPunctTokenizer()
stop_wrd = stopwords.words("english")
corpus = nltk.sent_tokenize(text)
print(len(corpus))
corpus[890]

11245


'“Hi, Brij esh.'

In [16]:
def normalise_document(doc):
    doc = re.sub(r"^a-zA-Z\s","",doc,re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    
    words = tknizer.tokenize(doc)
#     print(words[5:10])
    fil_words = [w for w in words if w not in stop_wrd]
    
    doc = " ".join(fil_words)
    return doc

In [17]:
norm_doc = np.vectorize(normalise_document) #it only transfer your functino in numpy for broad casting purpose
doc  = norm_doc(corpus)

In [18]:
doc = [sent for sent in doc if len(sent.split())>2]
print(type(doc))
print(len(doc))
doc[890]

<class 'list'>
10299


'phone buzzed .'

## Corpus Vocubulary

In [19]:
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence

Using TensorFlow backend.


In [20]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(doc)
word2id = tokenizer.word_index

In [21]:
print(len(word2id.keys()))

6556


In [22]:
text.text_to_word_sequence(doc[520])

['set',
 'routines',
 'protocols',
 'tools',
 'building',
 'software',
 'applications']

In [23]:
word2id["pad"] = 0
id2word = dict([(v,k) for k , v in word2id.items()])
wids = [[word2id[w] for w in text.text_to_word_sequence(d)] for d in doc]

vocab_size = len(word2id)
emb_size = 100
win_size = 1

print(vocab_size)
print(list(word2id.items())[5])
print(list(id2word.items())[5])
print(list(wids)[8])

6556
('like', 6)
(6, 'like')
[1028, 80, 478, 1850, 368, 2391, 1851, 350, 858, 1150, 1852, 3491, 142, 602, 2392, 23, 2387]


## generate context word

In [24]:
def generate_context_pair(corpus , wind_size , vocab_size ):
    for words in corpus:
        for idx , word in enumerate(words):
            cnt_word = []
            label_word = []
            start = idx - wind_size
            end = idx +wind_size + 1
            
            cnt_word.append([words[i] for i in range(start,end) if i!= idx if 0 <= i < len(words)])
            
            label_word.append(word)
            
            x = sequence.pad_sequences(cnt_word,maxlen = wind_size * 2)
            y = np_utils.to_categorical(label_word,vocab_size)
            yield(x , y)

In [25]:
# x = generate_context_pair(wids,2,vocab_size=vocab_size)
cnt = 0
for x ,y in generate_context_pair(wids,2,vocab_size):
    if 0 not in x[0]:
        print("*"*58)
        print("shape of training data --> ",x.shape, y.shape)
#         print(i[0]
        
        id_ = np.argwhere(y[0])
        print("Return the idx of non zero value -",id_," <=> ",id_[0]," <=> ",id_[0][0])
        print([id2word[id] for id in x[0]],"-->",id2word[id_[0][0]])
    cnt+=1
    if cnt == 5:
        break

**********************************************************
shape of training data -->  (1, 4) (1, 6556)
Return the idx of non zero value - [[60]]  <=>  [60]  <=>  60
['one', 'indian', 'chetan', 'bhagat'] --> girl
**********************************************************
shape of training data -->  (1, 4) (1, 6556)
Return the idx of non zero value - [[1028]]  <=>  [1028]  <=>  1028
['indian', 'girl', 'bhagat', 'author'] --> chetan
**********************************************************
shape of training data -->  (1, 4) (1, 6556)
Return the idx of non zero value - [[945]]  <=>  [945]  <=>  945
['girl', 'chetan', 'author', 'six'] --> bhagat


## np.sum vs k.sum2

In [26]:
def compareK_np_sum(x):
        print()
        print("^"*60)
        print("X --> ",x)
        print("sum along axis 0 (np) -->",np.sum(x,axis = 0,keepdims = True))
        sum_ = K.sum(x,axis = 0 ,keepdims = True)
        print(sum_)
        print("sum along axis 1 (np) -->",np.sum(x,axis = 1,keepdims = True))
        sum__= K.sum(x,axis = 1 ,keepdims = True)
        print(sum__)
        print("^"*60)
        del sum_ ,sum__
        print()

In [27]:
x_ = np.array([[2,3],[4,3]])
print(x_.shape)
compareK_np_sum(x_)

(2, 2)

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
X -->  [[2 3]
 [4 3]]
sum along axis 0 (np) --> [[6 6]]


NameError: name 'K' is not defined

## Model Architecture

In [28]:
import keras.backend as K
from keras.layers import Dense , Embedding , Lambda
from keras.models import Sequential

In [29]:
cbow = Sequential()

cbow.add(Embedding(input_dim = vocab_size,output_dim = emb_size ,input_length = win_size * 2))
cbow.add(Lambda(lambda x : K.mean(x , axis = 1)))
cbow.add(Dense(vocab_size,activation = "softmax"))

print(cbow.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 100)            655600    
_________________________________________________________________
lambda_1 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6556)              662156    
Total params: 1,317,756
Trainable params: 1,317,756
Non-trainable params: 0
_________________________________________________________________
None


## Train Model

In [49]:
cbow.compile(loss = "categorical_crossentropy",optimizer = "rmsprop")

In [50]:
for epoch in range(1,5):
    loss = 0
    cnt = 0
    for x,y in generate_context_pair(wids,win_size,vocab_size):
        cnt+=1
#         print(loss)
        loss += cbow.train_on_batch(x,y)
        
        if cnt > 5000:
            break
        
    print("epoch",epoch , "\tloss",loss)
    print()

epoch 1 	loss 45830.77659081883

epoch 2 	loss 46036.16768140702

epoch 3 	loss 46242.21440684785

epoch 4 	loss 46018.432844458104



In [54]:
weights = cbow.get_weights()[0]
weights = weights[1:]

print(weights.shape)

import pandas as pd
wrd_emb = pd.DataFrame(weights , index = list(id2word.values())[1:])
wrd_emb.head(6)

(6555, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
‘,0.281076,-0.181313,0.253112,-0.122151,0.152994,-0.095516,-0.029143,0.377922,0.280936,-0.166578,...,-0.027879,-0.259911,-0.394983,-0.462198,-0.138003,-0.036760,0.360835,0.145279,-0.347896,-0.358649
said,0.119032,-0.598810,0.103481,-0.323416,0.037999,0.091895,-0.107412,-0.134377,-0.170499,-0.045597,...,-0.846326,-0.249982,0.531245,0.477507,-0.057289,0.094422,0.100349,-0.397421,-0.038629,-0.398164
neel,-0.237389,-0.141943,-0.566874,-0.403153,-0.492787,-0.356656,0.072839,-0.228308,-0.123777,-0.424938,...,-0.072097,1.043128,-0.046427,0.276616,0.250141,0.135944,-0.296210,0.035597,-0.020644,-0.271495
debu,-0.265364,-0.161909,-0.280103,-0.210716,-0.160269,0.242691,-0.019847,0.310536,-0.126590,-0.137395,...,-0.122794,-0.123167,-0.304177,0.010149,-0.105604,0.100195,0.208448,-0.110721,0.093812,0.046361
like,0.029598,-0.215566,-0.159101,0.045574,-0.279257,-0.001587,-0.045344,0.047919,-0.217635,-0.028089,...,0.040511,0.226621,-0.286857,-0.070956,0.037725,0.028874,0.054807,-0.283012,0.012261,-0.113263
one,0.138026,-0.136071,-0.169180,-0.106101,0.049800,0.143565,-0.202722,-0.038723,-0.055654,-0.037901,...,-0.226241,-0.128330,-0.087331,-0.004598,-0.062975,0.081216,0.141682,-0.131175,-0.052328,0.050949
